In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import re
import time
from tqdm import tqdm

In [5]:
def best_weights(feature_df, results_df):
    num_features = len(feature_df.columns[2:-1])
    num_iterations = 500
    weights=[]
    accuracy_list = []
    weight_list = []
    weights_df = pd.DataFrame()
    results_range=results_df[results_df['Year'].isin(feature_df['Year'].unique())]
    # performance_dict={}
    # print(results_range)
    i=0
    for single_weight in tqdm(range(num_iterations)):
        results_index=[]
        temp_df=pd.DataFrame()
        comparison_column=[]
        weights = np.random.randint(low=-1, high=1, size=num_features).astype(np.float32)
        # weights = np.random.random(num_features)
        # weights /= np.sum(weights)
        fran_weights = feature_df.iloc[:, 2:-3].T.apply(lambda x: x * weights[:-2])
        fran_weights = fran_weights.T
        fran_weights = fran_weights.multiply(feature_df['Strength of Schedule']*weights[-2], axis=0).multiply(feature_df['Last 10 Rating']*weights[-1], axis=0)
        # print(fran_weights)
        # fran_weights = fran_weights.drop(['Strength of Schedule', 'Last 10 Rating'], axis=1)
        fran_weights['Scaled Sum'] = fran_weights.sum(axis=1)
        fran_weights.insert(0,"Team",feature_df['Team'].values)
        fran_weights.insert(len(fran_weights.columns), "Year", feature_df['Year'].values)
    
        for index, row in results_range.iterrows():
            # print(row['Year'])
            # print(index, row)
            team_1_weight = fran_weights[(fran_weights['Team']==row['Team 1']) & (fran_weights['Year']==row['Year'])]['Scaled Sum'].values
            # print(team_1_weight)
            team_2_weight = fran_weights[(fran_weights['Team']==row['Team 2']) & (fran_weights['Year']==row['Year'])]['Scaled Sum'].values
            if team_1_weight > team_2_weight:
                results_index.append(row['Team 1'])
            else:
                results_index.append(row['Team 2'])
        temp_df['Winner'] = results_range['Winner']
        temp_df['Predicted Winner'] = results_index
        # print(temp_df)
        comparison_column = np.where(temp_df["Winner"] == temp_df["Predicted Winner"], 1, 0)
        # print(sum(comparison_column))
        accuracy = (sum(comparison_column)/temp_df.shape[0])*100
        # print(performance_dict)
        # print(len(performance_dict))
        # print(accuracy, )
        accuracy_list.append(accuracy)
        weight_list.append(weights)
        i+=1
        pass
    weights_df['Accuracy'] = accuracy_list
    weights_df['Weights'] = weight_list
        
    
        
    return(weights_df)


    


In [6]:
features_df = pd.read_csv("data/2012-2025_MBB_Historical-Data.csv")
results_df = pd.read_csv("data/Bracket_Historical_Data.csv")
# print(results_df, features_df)
weights_df = best_weights(features_df, results_df)
print(weights_df)

  0%|          | 0/500 [00:00<?, ?it/s]/var/folders/f6/vt6ygq7j4ydbwfb99mfdjbnc0000gn/T/ipykernel_12538/208144470.py:34: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if team_1_weight > team_2_weight:
  0%|          | 1/500 [00:01<09:29,  1.14s/it]/var/folders/f6/vt6ygq7j4ydbwfb99mfdjbnc0000gn/T/ipykernel_12538/208144470.py:34: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if team_1_weight > team_2_weight:
  0%|          | 2/500 [00:02<09:44,  1.17s/it]/var/folders/f6/vt6ygq7j4ydbwfb99mfdjbnc0000gn/T/ipykernel_12538/208144470.py:34: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is

      Accuracy                                            Weights
0    44.458281  [2.0, 4.0, 7.0, 0.0, 1.0, 2.0, 9.0, 3.0, 6.0, ...
1     0.000000  [3.0, 9.0, 8.0, 8.0, 4.0, 8.0, 4.0, 9.0, 0.0, ...
2    44.956413  [6.0, 1.0, 8.0, 0.0, 0.0, 8.0, 3.0, 5.0, 3.0, ...
3    44.333748  [2.0, 7.0, 9.0, 8.0, 5.0, 2.0, 5.0, 1.0, 7.0, ...
4    44.956413  [1.0, 9.0, 3.0, 2.0, 6.0, 7.0, 7.0, 8.0, 1.0, ...
..         ...                                                ...
495  44.084682  [8.0, 1.0, 0.0, 2.0, 4.0, 5.0, 8.0, 5.0, 1.0, ...
496  44.831880  [8.0, 9.0, 1.0, 6.0, 0.0, 5.0, 7.0, 3.0, 9.0, ...
497  44.458281  [8.0, 3.0, 6.0, 6.0, 5.0, 2.0, 4.0, 6.0, 2.0, ...
498  45.330012  [0.0, 1.0, 3.0, 3.0, 1.0, 8.0, 7.0, 1.0, 4.0, ...
499  44.333748  [2.0, 7.0, 5.0, 2.0, 6.0, 8.0, 5.0, 0.0, 9.0, ...

[500 rows x 2 columns]


In [7]:
weights_df = weights_df.sort_values(by=['Accuracy'], ascending=False)

In [8]:
print(weights_df)

      Accuracy                                            Weights
39   45.579078  [4.0, 8.0, 1.0, 4.0, 1.0, 8.0, 8.0, 3.0, 3.0, ...
383  45.454545  [7.0, 4.0, 3.0, 9.0, 1.0, 2.0, 5.0, 6.0, 4.0, ...
345  45.454545  [6.0, 5.0, 1.0, 3.0, 0.0, 5.0, 4.0, 5.0, 3.0, ...
370  45.454545  [5.0, 8.0, 0.0, 4.0, 0.0, 6.0, 9.0, 9.0, 3.0, ...
473  45.330012  [7.0, 1.0, 4.0, 5.0, 1.0, 7.0, 8.0, 2.0, 1.0, ...
..         ...                                                ...
419   0.000000  [0.0, 7.0, 8.0, 0.0, 1.0, 0.0, 5.0, 5.0, 0.0, ...
293   0.000000  [7.0, 1.0, 9.0, 0.0, 6.0, 9.0, 4.0, 3.0, 0.0, ...
297   0.000000  [4.0, 2.0, 5.0, 3.0, 0.0, 0.0, 4.0, 1.0, 5.0, ...
415   0.000000  [4.0, 3.0, 6.0, 7.0, 9.0, 7.0, 5.0, 3.0, 7.0, ...
356   0.000000  [8.0, 6.0, 9.0, 0.0, 5.0, 0.0, 4.0, 3.0, 0.0, ...

[500 rows x 2 columns]


In [10]:
weights = weights_df['Weights'][0]
results_index = 0
# print(features_df)
df = features_df[features_df['Year']==2025]
# print(df)
fran_weights = df.iloc[:, 2:-3].T.apply(lambda x: x * weights[:-2])
fran_weights=fran_weights.T
# fran_weights = fran_weights.multiply(df['Strength of Schedule']*weights[-2], axis=0).multiply(df['Last 10 Rating']*weights[-1], axis=0)
fran_weights['Scaled Sum'] = fran_weights.sum(axis=1)
fran_weights['Team'] = features_df['Team']
fran_weights['Year'] = features_df['Year']
fran_weights = fran_weights.sort_values(by=['Scaled Sum'], ascending=False)
fran_weights.to_csv('Prediction_sum.csv')
